# Polyps Segmentation Masks Generator

This notebook is designed to read the xml files contained within the Polyps dataset to create a dataset with segmentation labels for training SPADEGAN.

In [ ]:
!pip install opencv-python numpy lxml


In [ ]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
import shutil
import random

# Parameters
sample_size = 500  # Set the number of samples you want to randomly select

# Paths
image_folder = 'path/to/Image'
xml_folder = 'path/to/Annotation'
output_image_folder = 'path/to/Images'
output_mask_folder = 'path/to/Masks'

# Ensure output directories are empty
if os.path.exists(output_image_folder):
    shutil.rmtree(output_image_folder)
os.makedirs(output_image_folder)
print(f"Created new directory: {output_image_folder}")

if os.path.exists(output_mask_folder):
    shutil.rmtree(output_mask_folder)
os.makedirs(output_mask_folder)
print(f"Created new directory: {output_mask_folder}")

# Get a list of all XML files
xml_files = [f for f in os.listdir(xml_folder) if f.endswith('.xml')]

# Randomly sample the XML files
sampled_xml_files = random.sample(xml_files, min(sample_size, len(xml_files)))

# Function to create segmentation masks and save images
def process_file(xml_file):
    print(f"Processing file: {xml_file}")
    
    # Parse XML file
    tree = ET.parse(os.path.join(xml_folder, xml_file))
    root = tree.getroot()

    # Extract the base filename without extension
    base_filename = os.path.splitext(xml_file)[0]
    width = int(root.find('size/width').text)
    height = int(root.find('size/height').text)

    print(f"Creating mask for {base_filename} with dimensions ({width}x{height})")

    # Create an empty mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # Loop through each object in the XML
    for obj in root.findall('object'):
        # Get bounding box coordinates
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        print(f"Drawing bounding box: ({xmin}, {ymin}) to ({xmax}, {ymax})")
        
        # Draw the bounding box on the mask
        cv2.rectangle(mask, (xmin, ymin), (xmax, ymax), color=255, thickness=-1)

    # Save the mask with the same name as the original XML file, but in .png format
    mask_filename = os.path.join(output_mask_folder, base_filename + '.png')
    cv2.imwrite(mask_filename, mask)
    print(f"Mask saved as: {mask_filename}")

    # Load the corresponding image
    image_path = os.path.join(image_folder, base_filename + '.jpg')
    if not os.path.exists(image_path):
        image_path = os.path.join(image_folder, base_filename + '.png')
    
    image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        print(f"Warning: Image '{image_path}' could not be read. Skipping this file.")
        return

    # Save the image in PNG format in the output image folder
    output_image_filename = os.path.join(output_image_folder, base_filename + '.png')
    cv2.imwrite(output_image_filename, image)
    print(f"Image saved as: {output_image_filename}\n")

# Process each sampled file
print(f"Starting the process with {len(sampled_xml_files)} samples...\n")
for xml_file in sampled_xml_files:
    process_file(xml_file)

print("Dataset creation process completed.")
